In [8]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import sqlite3
import pandas as pd
import sys
sys.path.append("/Users/peter/PycharmProjects/depth/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
conn = sqlite3.connect("../data.db")
cur = conn.cursor()

In [10]:
protein_df_all = pd.read_sql_query("select * from proteins", conn)

In [11]:
protein_df = protein_df_all[(protein_df_all.method.str.match("x-ray")) & (protein_df_all.cov_trimmed == 100) & (protein_df_all.cov_total >= 80) & (protein_df_all.resolution <= 3.5) & (protein_df_all.length >= 30)]

In [12]:
ds_df = pd.read_sql_query("select * from (select distinct id from datasets) a join proteins using (id)", conn)

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import numpy as np
from numpy import vectorize
import math
from time import time

In [31]:
from mllib.retrievers import SQLRetriever
from mllib.features.binary import Bfactors
import io
from Bio import AlignIO

tf = Bfactors()
sql_retriever = SQLRetriever(conn, tf.query)

In [32]:
r = sql_retriever.transform("4qnd_A")
r[:10]

array([[39.49],
       [34.57],
       [26.44],
       [23.22],
       [17.75],
       [25.37],
       [21.06],
       [20.82],
       [19.79],
       [27.47]])

In [25]:
s = tf.fit(r).transform(r)
s[:10]

array([[1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.]])

In [26]:
i = tf.inverse_transform(s)
i[:10]

AttributeError: 'BfactorsBinary' object has no attribute 'inverse_transform'

In [27]:
ds_df.count()

id             252
pdb_id         252
chain          252
sequence       252
length         252
thickness      252
method         252
resolution     252
cov_total      252
cov_trimmed    252
dtype: int64

In [28]:
print(ds_df.thickness.mean(), ds_df.thickness.std())

15.130753968253968 0.9547827953200508


In [33]:
print(ds_df.describe())

           length   thickness  resolution   cov_total  cov_trimmed
count   252.00000  252.000000  252.000000  252.000000        252.0
mean    262.47619   15.130754    2.709762   96.348191        100.0
std     188.72703    0.954783    0.520341    4.432093          0.0
min      30.00000   11.500000    1.150000   80.000000        100.0
25%      94.75000   14.675000    2.300000   94.642714        100.0
50%     250.50000   15.000000    2.800000   97.797980        100.0
75%     388.50000   15.700000    3.100000  100.000000        100.0
max    1046.00000   19.900000    3.500000  100.000000        100.0
